In [8]:
%matplotlib inline
from matplotlib import pyplot as plt
from time import time
from utils import *
from os.path import join
from multiprocessing import Pool
import numpy as np
import itertools
import time as timeMod
from time import mktime
from datetime import datetime
import scipy.spatial
import scipy.spatial.distance as dist 
import python_speech_features as psf
import pickle
from skimage.measure import block_reduce

In [7]:
a = np.array([[[1,2],[3,4]], [[5,6],[7,8]]])
a.reshape(len(a), -1)

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [9]:
data_root = 'drumData/'
sr = 48000 # this is the samplerate initially used to load the samples
drumNames = pickle.load(open(data_root+'drumNames.pickle'))
drumLengths = pickle.load(open(data_root+'drumLengths.pickle'))
drumMFCCs = {}
for d in drumNames:
    %time drumSamples[d] = np.load(join(data_root, d+'_samples.npy'))
n_fft = 1024
hop_length = n_fft/4
use_logamp = False # boost the brightness of quiet sounds
reduce_rows = 10 # how many frequency bands to average into one
reduce_cols = 1 # how many time steps to average into one
crop_rows = 32 # limit how many frequency bands to use
crop_cols = 32 # limit how many time steps to use
limit = None # set this to 100 to only process 100 samples

CPU times: user 319 µs, sys: 230 ms, total: 231 ms
Wall time: 339 ms
CPU times: user 595 µs, sys: 21.2 ms, total: 21.8 ms
Wall time: 29.7 ms
CPU times: user 497 µs, sys: 114 ms, total: 115 ms
Wall time: 170 ms
CPU times: user 914 µs, sys: 60.8 ms, total: 61.7 ms
Wall time: 90.5 ms
CPU times: user 486 µs, sys: 6.94 ms, total: 7.42 ms
Wall time: 10.9 ms
CPU times: user 443 µs, sys: 11.1 ms, total: 11.5 ms
Wall time: 17 ms
CPU times: user 401 µs, sys: 33.8 ms, total: 34.2 ms
Wall time: 49.5 ms


In [13]:
window = np.hanning(n_fft)
def job(y):
    mfcc = psf.mfcc(y, samplerate=sr, winlen=len(y)/sr, winstep=1.0/sr*hop_length, numcep=22, nfft=n_fft, winfunc=np.hanning)  
    amp = np.abs(mfcc)
    if reduce_rows > 1 or reduce_cols > 1:
        amp = block_reduce(amp, (reduce_rows, reduce_cols), func=np.mean)
    if amp.shape[1] < crop_cols:
        amp = np.pad(amp, ((0, 0), (0, crop_cols-amp.shape[1])), 'constant')
    amp = amp[:crop_rows, :crop_cols]
   
    #are you supposed to normalize mfcc?
    amp -= amp.min()
    if amp.max() > 0:
        amp /= amp.max()
    return amp

for d in drumNames:
    pool = Pool()
    %time mfccs = pool.map(job, drumSamples[d][:limit])
    mfccs = np.asarray(mfccs).astype(np.float32)
    mfccs = mfccs.reshape(len(mfccs), -1)
    drumMFCCs[d] = mfccs
    print "generated finger print for", d, mfccs.shape

data = np.concatenate([drumMFCCs[d] for d in drumNames])

CPU times: user 298 ms, sys: 335 ms, total: 633 ms
Wall time: 2.8 s
 generated finger print for kick (5158, 160)
CPU times: user 27.2 ms, sys: 23.7 ms, total: 51 ms
Wall time: 251 ms
generated finger print for tom (422, 160)
CPU times: user 142 ms, sys: 118 ms, total: 260 ms
Wall time: 1.35 s
generated finger print for snare (2546, 160)
CPU times: user 86.7 ms, sys: 88.7 ms, total: 175 ms
Wall time: 745 ms
generated finger print for clap (1324, 160)
CPU times: user 14.8 ms, sys: 11.1 ms, total: 25.9 ms
Wall time: 125 ms
generated finger print for hi.hat (159, 160)
CPU times: user 18.2 ms, sys: 15 ms, total: 33.2 ms
Wall time: 155 ms
generated finger print for ride (228, 160)
CPU times: user 40.9 ms, sys: 36.2 ms, total: 77.1 ms
Wall time: 396 ms
generated finger print for crash (723, 160)


In [15]:
for d in drumNames:
    np.save(join(data_root, d+'_mfccs.npy'), drumMFCCs[d])
    print "saved", d+'_mfccs.npy'

saved kick_mfccs.npy
saved tom_mfccs.npy
saved snare_mfccs.npy
saved clap_mfccs.npy
saved hi.hat_mfccs.npy
saved ride_mfccs.npy
saved crash_mfccs.npy


### Common setup 

In [ ]:
# create point -> class hashmap
def getClassesPerSample(data):
    drumClasses = {}
    classIndex = 0
    for i in range(len(data)):
        if sum(drumLengths[0:classIndex+1]) <= i:
            classIndex += 1
        drumClasses[tuple(data[i])] = drumNames[classIndex]
    return drumClasses

drumClasses = getClassesPerSample(data)

numNeighbors = 10

## Arguments
## data -    an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i.
# numPerClass - Assumes that points from the same class are contiguous and in
#            the same order as drumNames. Should use drumLengths for this parameter
# calcFunc - The summary statistic you want to calcuate per class (mean, medain, etc)
def calculateFuncPerClass(data, numPerClass, calcFunc):
    segments = [0]+numPerClass
    for i in range(1, len(segments)):
        segments[i] = segments[i] + segments[i-1]
    valuesPerClass = []
    for i in range(len(segments)-1):
        valuesPerClass.append(calcFunc(data[segments[i]:segments[i+1]]))
    return valuesPerClass

### pairwise distance implementation

In [ ]:
#using https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.spatial.distance.cdist.html
#and argpartition from - https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array

## Arguments - pointData is m x n np array, with n points of m dimensions
# pointD numNeighbors is the number of nearest neighbors for which to compare classes
#
## returns - an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i
def calculateKNNClassAccuracy_pairwise(pointData, numNeighbors, printTimes=False):
    startTime = timeMod.time()
    pairwiseDist = dist.cdist(pointData, pointData)
    if printTimes:
        print "pairwise distances calculated", timeMod.time() - startTime
    kPartition = np.argpartition(pairwiseDist, -numNeighbors)
    if printTimes:
        print "partitions calculated", timeMod.time() - startTime
    fracSameNeighborClasses = np.zeros((len(data)))
    
    for i in range(len(pairwiseDist)):
        neighborIndexes = kPartition[i][-numNeighbors:]
        neighbors = [data[ind] for ind in neighborIndexes]
        
        sampleClass = drumClasses[tuple(data[i])]
        neighborClasses = [drumClasses[tuple(neighbor)] for neighbor in neighbors]
        numSameNeighborClasses = len(filter(lambda c : c == sampleClass, neighborClasses))
        fracSameNeighborClasses[i] = numSameNeighborClasses * 1.0 / numNeighbors
    
    if printTimes:
        print "knn classes calculated", timeMod.time() - startTime
        
    return fracSameNeighborClasses

#classAccuracies_LD = calculateKNNClassAccuracy(newData, numNeighbors, True)
classAccuracies_HD = calculateKNNClassAccuracy_pairwise(data, numNeighbors, True)